In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor)
from torch.autograd import Variable
from torch.nn import functional as F
from torchmetrics import Accuracy, Recall, Precision, Specificity, ConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_recall_curve, average_precision_score,roc_curve, auc, precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from collections import Counter
import random
import math
import torch.optim as optim
from tabulate import tabulate
from ray import tune

from fractions import Fraction

In [9]:
# !pip install torch -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
# Create Tensors to hold input and outputs.

#第一组参数
# beta_1=np.append(np.array([4,4,-8,-4,6,8,-4,4,-6,-8]),np.zeros(90))
# beta_2=np.append(np.array([-2,2,4,-2,3,-4,2,-2,-3,4]),np.zeros(90))
# beta_3=np.append(np.array([-3,-3,6,-3,-4.5,-6,3,-3,4.5,6]),np.zeros(90))

#第二组参数

beta_1=np.append(np.array([4,4,-8,-4,6,-4,8,4,-6,-8]),np.zeros(90))
beta_2=np.append(np.array([-2,2,4,-2,3,2,-4,-2,-3,4]),np.zeros(90))
beta_3=np.append(np.array([-3,-3,6,-3,-4.5,3,-6,-3,4.5,6]),np.zeros(90))

beta_1=beta_1.reshape(-1,1)
beta_2=beta_2.reshape(-1,1)
beta_3=beta_3.reshape(-1,1)

n1=200
n2=200
n3=200
p=100
corval=0.5
prior=0

def sigmoid(X):
    return .5 * (1 + np.tanh(.5 * X))

def generate_data(corval,beta,n,p):
    mean=np.zeros(p)
    sigma=np.array([[corval**abs(i-j) for i in range(p)] for j in range(p)])
    x=torch.tensor(np.random.multivariate_normal(mean=mean,cov=sigma,size=n))
    Pi_test= sigmoid(x@beta)
    y=np.random.binomial(1,Pi_test.ravel(),n)
    x=x.to(torch.float32)
    #y=torch.tensor(y,dtype=torch.double)
    y=torch.tensor(y,dtype=torch.float)
    #y=y.to(torch.float32)
    return x,y

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    
def cubic_root(x):
    return math.copysign(math.pow(abs(x), 1.0/3.0), x)

In [3]:
################## first step##################   
def train_model(config):
    split_num=5
    kf = KFold(n_splits=split_num)


    sum_tGM=0
    sum_test_BCE_loss=0
    for idx_train,idx_test in kf.split(inputs_1):
        lambda1,lambda2,lr= config["lambda1"], config["lambda2"],config["lr"]
        model_1=MLP(seed=1)
        model_2=MLP(seed=1)
        model_3=MLP(seed=1)
        max_iteration=1000
        learning_rate=lr
        params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
        optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)
        loss_fn = torch.nn.BCELoss()
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

        BCE=[]

        for t in range(max_iteration):
            optimizer.zero_grad() # renew optimizer
            out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1[idx_train])
            out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2[idx_train])
            out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3[idx_train])# forward propagate

            # extract parameters
            #[:-1] for leaving out bias term#
            model_1_all_linear1_params = model_1.linear1
            model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
            model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
            model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])
           

            model_2_all_linear1_params = model_1.linear1
            model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
            model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
            model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])
    

            model_3_all_linear1_params = model_3.linear1
            model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
            model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
            model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])
    


            # compute loss
            all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
            all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                             model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                             model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

            BCE_loss_1 = loss_fn(out_1, targets_1[idx_train].reshape(-1,1))
            BCE_loss_2 = loss_fn(out_2, targets_2[idx_train].reshape(-1,1))
            BCE_loss_3 = loss_fn(out_3, targets_3[idx_train].reshape(-1,1))

#             l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)
            l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

            l2_regularization = lambda2* torch.sum(all_linear234_params.pow(2))
            BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
            loss =BCE_loss+l1_regularization +l2_regularization

            # record loss
            BCE.append(BCE_loss.item())

            # compute derivative
            loss.backward()

            # gradient descent
            optimizer.step()

            # learning rate decay
            scheduler.step()
        
        
#         test_out_1, test_layer1_out_1, test_layer2_out_1,test_layer3_out_1= model_1(train_x1[idx_test])
#         test_out_2, test_layer1_out_2, test_layer2_out_2,test_layer3_out_2= model_2(train_x2[idx_test])
#         test_out_3, test_layer1_out_3, test_layer2_out_3,test_layer3_out_3= model_3(train_x3[idx_test])
        
#         test_BCE_loss_1 = loss_fn(test_out_1, targets_1[idx_test].reshape(-1,1))
#         test_BCE_loss_2 = loss_fn(test_out_2, targets_2[idx_test].reshape(-1,1))
#         test_BCE_loss_3 = loss_fn(test_out_3, targets_3[idx_test].reshape(-1,1))
        
#         test_BCE_loss=test_BCE_loss_1+test_BCE_loss_2+test_BCE_loss_3
        
#         sum_test_BCE_loss+=test_BCE_loss.item()

#     #print("loss: ",)
#     tune.report(my_test_BCE_loss=sum_test_BCE_loss/split_num)

        test_prediction_1= (model_1(train_x1[idx_test])[0]>0.5).clone().int()
        test_target_1= train_y1[idx_test].reshape(-1,1).int()

        test_prediction_2= (model_2(train_x2[idx_test])[0]>0.5).clone().int()
        test_target_2= train_y2[idx_test].reshape(-1,1).int()

        test_prediction_3= (model_3(train_x3[idx_test])[0]>0.5).clone().int()
        test_target_3= train_y3[idx_test].reshape(-1,1).int()

        test_prediction=np.append(test_prediction_1,test_prediction_2)
        test_prediction=np.append(test_prediction,test_prediction_3)
        test_prediction=test_prediction.tolist()

        test_target=np.append(test_target_1,test_target_2)
        test_target=np.append(test_target,test_target_3)
        test_target=test_target.tolist()

        final_matrix=confusion_matrix(test_target,test_prediction)
        tTPR=recall_score(test_target,test_target)#TPR
        tTNR=final_matrix[0,0]/(sum(final_matrix[0,:]))#TNR
        tGM=math.sqrt(tTPR*tTNR)

        sum_tGM+=tGM

    tune.report(my_test_sum_tGM=sum_tGM/split_num)


In [4]:
TPR=[]
TNR=[]
GM=[]
accuracy=[]
precision=[]
f1=[]=[]
vsSEN=[]
vsSPE=[]
vsGM=[]
vsMR=[]
vsCCR=[]


vs = np.zeros((1,100))

In [5]:
for i in range(10):
    setup_seed(i)
    x1,y1=generate_data(corval,beta_1,n1,p)
    x2,y2=generate_data(corval,beta_2,n2,p)
    x3,y3=generate_data(corval,beta_3,n3,p)
    
#     print(sigmoid(x1@beta_1).squeeze().cov())

    train_x1, test_x1, train_y1, test_y1= train_test_split(x1,y1, test_size=0.2, random_state=1)
    train_x2, test_x2, train_y2, test_y2= train_test_split(x2,y2, test_size=0.2, random_state=1)
    train_x3, test_x3, train_y3, test_y3= train_test_split(x3,y3, test_size=0.2, random_state=1)
    
     # switch training set
    inputs_1,targets_1 = train_x1,train_y1
    inputs_2,targets_2 = train_x2,train_y2
    inputs_3,targets_3 = train_x3,train_y3

    # switching testing set
    test_inputs_1, test_targets_1 = test_x1,test_y1
    test_inputs_2, test_targets_2 = test_x2,test_y2
    test_inputs_3, test_targets_3 = test_x3,test_y3
    
    
################## first step##################   
    # choose lambda1,lambda2 by ray tune
    class MLP(torch.nn.Module):
        def __init__(self,seed):
            super(MLP, self).__init__()
            torch.manual_seed(seed)
            self.linear1 = torch.nn.Parameter(torch.randn(p))
            self.linear2 = torch.nn.Linear(p,10)
            self.linear3 = torch.nn.Linear(10,10)
            self.linear4 = torch.nn.Linear(10,1)

        def forward(self, x):
            layer1_out = self.linear1*x
            layer2_out = F.relu(self.linear2(layer1_out))
            layer3_out = F.relu(self.linear3(layer2_out))
            out= torch.sigmoid(self.linear4(layer3_out))
            return out, layer1_out, layer2_out,layer3_out
    
    config={
        "lambda1": tune.choice([0.03,0.04,0.05]),
        "lambda2": tune.choice([0.001,0.002,0.003]),
        "lr":tune.choice([0.6,0.7,0.8])
    }

    result = tune.run(
        train_model,  
        config=config,
        num_samples=30)
    
################## second step##################   

    final_parameter=result.get_best_config('my_test_sum_tGM',mode='max')
    lambda1,lambda2,lr=final_parameter["lambda1"],final_parameter["lambda2"],final_parameter["lr"]
    learning_rate=lr
    max_iteration=1000
    
    BCE=[]

    model_1=MLP(seed=1)
    model_2=MLP(seed=1)
    model_3=MLP(seed=1)

    loss_fn = torch.nn.BCELoss()
    params_to_optimize=list(model_1.parameters())+list(model_2.parameters())+list(model_3.parameters())
    optimizer = torch.optim.SGD(params_to_optimize, lr=learning_rate)

# learning rate decay scheme
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300,400,500,600,700,800], gamma=0.3)

    loss_record=[]
# loop for max_iteration times
    for t in range(max_iteration):

        # renew optimizer
        optimizer.zero_grad()

        # forward propagate
        out_1, layer1_out_1, layer2_out_1,layer3_out_1= model_1(inputs_1)
        out_2, layer1_out_2, layer2_out_2,layer3_out_2= model_2(inputs_2)
        out_3, layer1_out_3, layer2_out_3,layer3_out_3= model_3(inputs_3)



        # extract parameters
        #[:-1] for leaving out bias term#
        model_1_all_linear1_params = model_1.linear1
        model_1_all_linear2_params= torch.cat([x.view(-1) for x in model_1.linear2.parameters()][:-1])
        model_1_all_linear3_params= torch.cat([x.view(-1) for x in model_1.linear3.parameters()][:-1])
        model_1_all_linear4_params= torch.cat([x.view(-1) for x in model_1.linear4.parameters()][:-1])

        model_2_all_linear1_params = model_1.linear1
        model_2_all_linear2_params= torch.cat([x.view(-1) for x in model_2.linear2.parameters()][:-1])
        model_2_all_linear3_params= torch.cat([x.view(-1) for x in model_2.linear3.parameters()][:-1])
        model_2_all_linear4_params= torch.cat([x.view(-1) for x in model_2.linear4.parameters()][:-1])

        model_3_all_linear1_params = model_3.linear1
        model_3_all_linear2_params= torch.cat([x.view(-1) for x in model_3.linear2.parameters()][:-1])
        model_3_all_linear3_params= torch.cat([x.view(-1) for x in model_3.linear3.parameters()][:-1])
        model_3_all_linear4_params= torch.cat([x.view(-1) for x in model_3.linear4.parameters()][:-1])


        # compute loss

        all_linear1_params=torch.cat(( model_1_all_linear1_params.reshape(1,-1),  model_2_all_linear1_params.reshape(1,-1), model_3_all_linear1_params.reshape(1,-1)),0)
        all_linear234_params=torch.cat((model_1_all_linear2_params,model_1_all_linear3_params, model_1_all_linear4_params,
                                         model_2_all_linear2_params,model_2_all_linear3_params, model_2_all_linear4_params,
                                         model_3_all_linear2_params,model_3_all_linear3_params,model_3_all_linear4_params),0)

        BCE_loss_1 = loss_fn(out_1, targets_1.reshape(-1,1))
        BCE_loss_2 = loss_fn(out_2, targets_2.reshape(-1,1))
        BCE_loss_3 = loss_fn(out_3, targets_3.reshape(-1,1))

    #     l1_regularization = lambda1 * smooth_l1(all_linear1_params,prior)

        l1_regularization = lambda1 * torch.sum(torch.norm(all_linear1_params[:,prior:],p=2,dim=0))

        l2_regularization = lambda2 * torch.sum(all_linear234_params.pow(2))
        BCE_loss=BCE_loss_1+BCE_loss_2+BCE_loss_3
        loss =BCE_loss+l1_regularization +l2_regularization

        loss_record.append(loss.item())
        # record loss
        BCE.append(BCE_loss.item())

        # compute derivative
        loss.backward()

        # gradient descent
        optimizer.step()

        # learning rate decay
        scheduler.step()
        
##################prediction ##################   

    prediction_1= (model_1(test_x1)[0]>0.5).clone().int()
    target_1= test_y1.reshape(-1,1).int()

    prediction_2= (model_2(test_x2)[0]>0.5).clone().int()
    target_2= test_y2.reshape(-1,1).int()

    prediction_3= (model_3(test_x3)[0]>0.5).clone().int()
    target_3= test_y3.reshape(-1,1).int()

    prediction=np.append(prediction_1,prediction_2)
    prediction=np.append(prediction,prediction_3)
    prediction=prediction.tolist()

    target=np.append(target_1,target_2)
    target=np.append(target,target_3)
    target=target.tolist()

    final_matrix=confusion_matrix(target,prediction)
    TPR.append(recall_score(target,prediction))#TPR
    TNR.append(final_matrix[0,0]/(sum(final_matrix[0,:])))#TNR
    GM.append(math.sqrt(TPR[-1]*TNR[-1]))
    accuracy.append(accuracy_score(target,prediction))
    precision.append(precision_score(target,prediction))
    f1.append(f1_score(target,prediction))
    
    
################## variable selection ##################   
    model_1_weight=model_1_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_2_weight=model_2_all_linear1_params.detach().numpy().copy().reshape(1,-1)
    model_3_weight=model_3_all_linear1_params.detach().numpy().copy().reshape(1,-1)

    threshold=0.12
    mcl_w=np.zeros(p)

    for j in range(p):
        mcl_w[j]=cubic_root(model_1_weight[0,j]*model_2_weight[0,j]*model_3_weight[0,j])

    max_mcl_w=max(abs(mcl_w))
    for j in range(p):
        if (abs(mcl_w[j]))<=threshold*max_mcl_w:
            model_1_weight[0,j]=0
            model_2_weight[0,j]=0
            model_3_weight[0,j]=0
        else:
            model_1_weight[0,j]=model_1_weight[0,j]
            model_2_weight[0,j]=model_2_weight[0,j]
            model_3_weight[0,j]=model_3_weight[0,j]
    
    index=np.nonzero(model_1_weight)[1]
    
    vs[:,index]+=1
    
    beta=np.array(beta_1)
    trueindex=np.where(beta!=0)[0]
    trueindex=trueindex.tolist()
    trueindex=set(trueindex)
    
    index=np.nonzero(model_1_weight)[1]
    index=index.tolist()
    index=set(index)
    
    TP=len(trueindex.intersection(index))
    FP=len(index.difference(trueindex))
    FN=len(trueindex)-TP
    TN=p-len(trueindex)-FP
    print(TP)
    print(FP)
    vsSEN.append(TP/(TP+FN))
    vsSPE.append(TN/(TN+FP))
    vsGM.append(math.sqrt(vsSEN[-1]*vsSPE[-1]))
    vsMR.append((FP+FN)/(TP+FN+TN+FP))
    vsCCR.append(1-vsMR[-1])
    
    print(i)

2022-11-15 16:03:58,988	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2022-11-15 16:03:59,925	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,lambda1,lambda2,lr,iter,total time (s),my_test_sum_tGM
train_model_0fc40_00000,TERMINATED,10.224.0.12:11415,0.05,0.001,0.8,1,13.4586,0.87563
train_model_0fc40_00001,TERMINATED,10.224.0.12:11441,0.03,0.003,0.6,1,14.6187,0.841642
train_model_0fc40_00002,TERMINATED,10.224.0.12:11443,0.03,0.003,0.6,1,13.7487,0.841642
train_model_0fc40_00003,TERMINATED,10.224.0.12:11445,0.05,0.001,0.7,1,15.7233,0.882189
train_model_0fc40_00004,TERMINATED,10.224.0.12:11447,0.05,0.003,0.7,1,14.1593,0.887147
train_model_0fc40_00005,TERMINATED,10.224.0.12:11449,0.04,0.002,0.7,1,13.7752,0.86599
train_model_0fc40_00006,TERMINATED,10.224.0.12:11451,0.04,0.002,0.8,1,14.536,0.872888
train_model_0fc40_00007,TERMINATED,10.224.0.12:11452,0.03,0.002,0.7,1,14.5978,0.858644
train_model_0fc40_00008,TERMINATED,10.224.0.12:11455,0.04,0.002,0.7,1,14.4515,0.86599
train_model_0fc40_00009,TERMINATED,10.224.0.12:11457,0.04,0.002,0.6,1,14.5354,0.861082


(raylet) [2022-11-15 16:04:08,883 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 319672320; capacity: 29394726912. Object creation will fail if spilling is required.


Result for train_model_0fc40_00000:
  date: 2022-11-15_16-04-15
  done: false
  experiment_id: 890de6c86284436f95830277546d3436
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8756299004831753
  node_ip: 10.224.0.12
  pid: 11415
  time_since_restore: 13.458627462387085
  time_this_iter_s: 13.458627462387085
  time_total_s: 13.458627462387085
  timestamp: 1668499455
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0fc40_00000
  warmup_time: 0.003400564193725586
  
Result for train_model_0fc40_00000:
  date: 2022-11-15_16-04-15
  done: true
  experiment_id: 890de6c86284436f95830277546d3436
  experiment_tag: 0_lambda1=0.0500,lambda2=0.0010,lr=0.8000
  hostname: rd-PowerEdge-R940xa
  iterations_since_restore: 1
  my_test_sum_tGM: 0.8756299004831753
  node_ip: 10.224.0.12
  pid: 11415
  time_since_restore: 13.458627462387085
  time_this_iter_s: 13.458627462387085
  time_total_s: 13.458627462387085
  timestamp: 1668499455
  timesteps_since_r

(raylet) [2022-11-15 16:04:18,889 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 319574016; capacity: 29394726912. Object creation will fail if spilling is required.


...

2022-11-15 16:07:43,933	INFO tune.py:758 -- Total run time: 21.08 seconds (20.90 seconds for the tuning loop).


7
4
9


(raylet) [2022-11-15 16:07:49,073 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 237510656; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:07:59,083 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 237170688; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:08:09,092 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 237129728; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:08:19,102 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 237129728; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:12:59,377 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236494848; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:13:09,385 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236462080; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:13:19,394 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236457984; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:13:29,402 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236449792; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:18:09,675 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236011520; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:18:19,685 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236011520; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:18:29,695 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236003328; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:18:39,704 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 236003328; capacity: 29394726912. Object creation will fail if sp

In [6]:
from numpy import *
mean(TPR)
mean(TNR)
mean(GM)
mean(accuracy)
mean(precision)
mean(f1)

0.7935099665058403

0.8145829101136035

0.8030801264300947

0.8041666666666668

0.8287356226428312

0.8097012065647086

(raylet) [2022-11-15 16:21:49,889 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 234213376; capacity: 29394726912. Object creation will fail if spilling is required.


In [7]:
mean(vsSEN)
mean(vsSPE)
mean(vsGM)
mean(vsMR)
mean(vsCCR)

0.7700000000000001

0.9388888888888889

0.8492465897208298

0.078

0.9219999999999999

(raylet) [2022-11-15 16:21:59,895 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 234094592; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:22:09,903 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 233521152; capacity: 29394726912. Object creation will fail if spilling is required.


In [8]:
vs

array([[10.,  3., 10., 10.,  3.,  1., 10., 10., 10., 10.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         3.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  8.,  0.,  6.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  1.,  3.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  3.,
         0.,  2.,  0.,  5.,  0.,  0.,  0.,  2.,  0.]])

(raylet) [2022-11-15 16:22:19,912 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 233521152; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:22:29,922 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 233500672; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:22:39,931 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 233476096; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:22:49,940 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 233463808; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:27:30,197 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232861696; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:27:40,207 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232824832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:27:50,216 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232824832; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:28:00,226 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232779776; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:32:40,491 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232185856; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:32:50,501 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232173568; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:33:00,511 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 232140800; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:33:10,521 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 231448576; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:37:50,779 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229912576; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:38:00,788 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229863424; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:38:10,797 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229830656; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:38:20,806 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229826560; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:43:01,081 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229257216; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:43:11,091 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229220352; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:43:21,101 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229216256; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:43:31,111 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 229208064; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:48:11,371 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 228589568; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:48:21,381 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 228581376; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:48:31,391 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 228577280; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:48:41,400 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 228536320; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:53:21,659 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227954688; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:53:31,668 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227946496; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:53:41,678 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227921920; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:53:51,687 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227909632; capacity: 29394726912. Object creation will fail if sp

(raylet) [2022-11-15 16:58:31,956 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227323904; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:58:41,965 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227282944; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:58:51,975 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227278848; capacity: 29394726912. Object creation will fail if spilling is required.
(raylet) [2022-11-15 16:59:01,984 E 10510 10528] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-11-15_16-03-56_955441_8104 is over 95% full, available space: 227233792; capacity: 29394726912. Object creation will fail if sp

In [52]:
vsSPE

[1.0,
 0.9555555555555556,
 0.9777777777777777,
 0.9666666666666667,
 0.9888888888888889,
 1.0,
 0.9888888888888889,
 0.9888888888888889,
 0.9777777777777777,
 0.9777777777777777]